In [181]:
import random
import pandas as pd

### Find extrema(min, max) of a function:
\begin{equation*}
\frac{x - 3}{(x + 5)} - (x + 3)(x - 5), x  \in [15, 45], x\not=5
\end{equation*}

In [182]:
def fun(x):
    assert x != 5
    return ((x - 3) / (x + 5)) - ((x + 3) * (x - 5))

def negative_fun(y):
    return -fun(y)

values = sorted([(fun(i), i) for i in range(15, 46)], 
                    key=lambda x: x[0], 
                    reverse=True)
actual_max, _ = values[0]
actual_min, _ = values[-1]
print(f'actual max = { actual_max }')
print(f'actual min = { actual_min }')

actual max = -179.4
actual min = -1919.16


### Util operations

In [183]:
def invert_bit(bits, i):
    x = list(bits)
    x[i] = int(not int(x[i]))
    return ''.join(map(str, x))

def fill_zeros(bits, length):
    b = list(bits)
    while len(b) != length:
        b.insert(0, 0)
    return ''.join(map(str, b))

### Chromosome representation

In [184]:
def chromosome_bits(n, bits_len=None):
    if bits_len:
        return fill_zeros(bin(n)[2:], bits_len)
    return bin(n)[2:]
    
def phenotype(n):
    return int(n, base=2)

assert chromosome_bits(10) == '1010'
assert chromosome_bits(10, bits_len=5) == '01010'
assert phenotype('1010') == 10
assert phenotype(chromosome_bits(1234)) == 1234
assert chromosome_bits(45) == '101101'
assert phenotype('101101') == 45
assert phenotype(chromosome_bits(45)) == 45

### Genetic operators

* Mutation(bit_position=3)
    - 000**0**00 >> 000**1**00

In [185]:
def mutation(chromosome, 
             probability=0.2, 
             bit_position=None):
    def mutate():
        if bit_position is None:
            pos = random.randint(0, len(chromosome) - 1)
        else:
            pos = bit_position
        return invert_bit(chromosome, pos)

    assert (0 <= probability <= 1)
    
    if random.random() < probability:
        return mutate()
    return chromosome

assert mutation('1111', bit_position=1, probability=0) == '1111'
assert mutation(chromosome_bits(15), bit_position=0, probability=1) == '0111'
assert mutation(chromosome_bits(15), bit_position=1, probability=1) == '1011'
assert mutation(chromosome_bits(15), bit_position=2, probability=1) == '1101'
assert mutation(chromosome_bits(15), bit_position=3, probability=1) == '1110'

def safe_mutation(chromosome, 
                  probability=0.2, 
                  bit_position=None,
                  allow_range=range(15, 46)):
    mutated = mutation(chromosome, probability, bit_position)
    bad_mutation_conditions = [
        phenotype(mutated) not in allow_range,
        fun(phenotype(mutated) < fun(phenotype(chromosome))) 
    ]
    if any(bad_mutation_conditions):
        return chromosome       
    return mutated

* Crossing
    * rift = 3
        - **000**000 >> 000 | 111 = 000111
        - **111**111 >> 111 | 000 = 111000
        
    * rift = 1
        - **1**111 >> 1 | 000 = 1000
        - **0**000 >> 0 | 111 = 0111

In [186]:
def crossing(chromosome1, chromosome2, rift=None, probability=0.8):
    def cross():
        assert len(chromosome1) == len(chromosome2)
        if rift is None:
            pos = random.randint(0, len(chromosome1))
        else:
            pos = rift
        assert 0 <= pos <= len(chromosome1)
        return (''.join([chromosome1[:pos], chromosome2[pos:]]),
                ''.join([chromosome2[:pos], chromosome1[pos:]]))
    
    assert (0 <= probability <= 1)
    
    if random.random() < probability:
        return cross()
    return chromosome1, chromosome2


assert crossing('000000', '111111', 3, 1) == ('000111', '111000')
assert crossing('1111', '0000', 1, 1) == ('1000', '0111')

def safe_crossing(chromosome1, 
                  chromosome2, 
                  rift=None, 
                  probability=0.8,
                  allow_range=range(15, 46)):
    c1, c2 = crossing(chromosome1, chromosome2, rift, probability)
    bad_crossing_conditions = [
        (phenotype(c1) not in allow_range),
        (phenotype(c2) not in allow_range),
        (fun(phenotype(c1) < fun(phenotype(chromosome1))) and
         fun(phenotype(c2) < fun(phenotype(chromosome2))))
    ]
    if any(bad_crossing_conditions):
        return chromosome1, chromosome2
    return c1, c2


def get_best_chromosome(c1, c2):
    return c1 if fun(phenotype(c1)) > fun(phenotype(c2)) else c2

### Genetic selection

In [187]:
def population_df(chromosomes):
        df = pd.DataFrame(data=[ch_i for ch_i in chromosomes], columns=['chromosome'])
        df['phenotype'] = df.apply(lambda x: phenotype(x['chromosome']), axis=1) 
        df['fun_value'] = df.apply(lambda x: fun(x['phenotype']), axis=1)
        df['sel_probability'] = df['fun_value'] / df['fun_value'].sum()
        assert round(df['sel_probability'].sum(), 5) == 1
        return df

def roulette(chromosomes, survive_probabilities):
    n = len(chromosomes)
    assert n == len(chromosomes) == len(survive_probabilities)
    s = sorted([(i, n * sp) 
                for i, sp in enumerate(survive_probabilities)],
               key=lambda x: x[1])
    
    survived = []
    s_rest = []

    for _ in range(n):
        i, ch_amount = s.pop()
        if round(ch_amount) > 0:
            survived.append([chromosomes[i]] * round(ch_amount))
        else:
            s_rest.append((i, ch_amount))

    survived = sum([i for i in survived if i], [])
    s_rest = sorted(s_rest, key=lambda x: x[1])
    while len(survived) != n:
        if len(survived) < n:
            i, ch_amount = s_rest.pop()
            survived.append(chromosomes[i])
        if len(survived) > n:
            i, ch_amount = s_rest.pop(0)
            survived.remove(chromosomes[i])
    return survived

def select_next_population(df):
    pass

### Initial values

In [188]:
# population contains N chromosomes 
# each chromosome consist of L bits
N = 4
L = len(chromosome_bits(45))

population = [
    chromosome_bits(15, L),
    chromosome_bits(32, L),
    chromosome_bits(27, L),
    chromosome_bits(18, L),
]

print('Initial population:')
print(population_df(population))
assert len(population) == N
assert L == 6

Initial population:
  chromosome  phenotype   fun_value  sel_probability
0     001111         15 -179.400000         0.087290
1     100000         32 -944.216216         0.459425
2     011011         27 -659.250000         0.320770
3     010010         18 -272.347826         0.132516


### Genetic algorithm

In [189]:
p_df = population_df(population)
print(roulette(p_df['chromosome'], p_df['sel_probability']))

['100000', '100000', '011011', '010010']
